# Backtest Example

In this notebook I will demonstrate an end to end cycle with adapted config based on llm feedback.
The scan and buy/sell rules are the same as the dynamic_scan_example.ipynb .

Buying criteria:
- Current month is an allowed month
- 10 ema and 21 ema sloping upward
- 10 ema above 21 ema

Selling criteria:
- Current month is an allowed month
- 10 ema and 21 ema sloping down
- 10 ema below 21 ema

In [21]:
from datetime import datetime
import backtrader as bt
import pandas
from backtrader.analyzers import AnnualReturn
import yfinance as yf
from stats.tradeLog import TradeLog
from stats.strategySummery import build_summary_report
import json
import os
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
import dotenv

dotenv.load_dotenv()

class SmaCross(bt.Strategy):
    params = dict(
        period_fast=10,
        period_slow=21
    )

    def __init__(self):
        self.sma_pairs = []

        i = 1
        for data in self.datas:
            sma10 = bt.ind.SMA(data, period=self.p.period_fast)
            sma21 = bt.ind.SMA(data, period=self.p.period_slow)
            months_to_trade = [i, i+6]
            self.sma_pairs.append((data, sma10, sma21, months_to_trade))
            i = i + 1

    def next(self):
        for data, sma10, sma21, months_to_trade in self.sma_pairs:
            if not len(data) > self.p.period_slow:
                continue

            current_date = self.data.datetime.date(0)
            is_positioned = self.getposition(data)

            sma10_above_sma21 = sma10[0] > sma21[0]
            sma10_sloping_up = sma10[0] > sma10[-1]
            sma21_sloping_up = sma21[0] > sma21[-1]

            if not is_positioned and current_date.month in months_to_trade:
                if sma10_above_sma21 and sma10_sloping_up and sma21_sloping_up:
                    self.buy(data=data, size=10)

            elif is_positioned:
                if not sma10_above_sma21 and not sma10_sloping_up and not sma21_sloping_up:
                    self.close(data=data)


stocks_pool = ["NVDA", "AAPL", "MSFT", "GOOG", "TSLA", "AMZN"]
# Cache data to avoid re-downloading in the loop
cached_data = {}
print("Downloading data...")
for stock in stocks_pool:
    df = yf.download(stock, start=datetime(2017, 1, 1), end=datetime(2024, 12, 31), progress=False)
    df.columns = ["Open", "High", "Low", "Close", "Volume"]
    cached_data[stock] = df
print("Data download complete.")

# --- LangGraph & Optimization Logic ---

class OptimizationState(TypedDict):
    params: dict
    report: str
    iteration: int
    max_iterations: int

def run_backtest_node(state: OptimizationState):
    """Runs the backtest with the current parameters in the state."""
    print(f"\n--- Running Backtest (Iter {state['iteration']}) with {state['params']} ---")
    
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(100000)
    cerebro.addsizer(bt.sizers.PercentSizer)

    # Load data from cache
    for stock, df in cached_data.items():
        stock_datafeed = bt.feeds.PandasData(dataname=df, name=stock)
        cerebro.adddata(stock_datafeed)

    # Add strategy with dynamic parameters
    cerebro.addstrategy(SmaCross, **state['params'])
    
    cerebro.addanalyzer(TradeLog, _name='trade_log')
    cerebro.addanalyzer(AnnualReturn)

    results = cerebro.run()
    strategy = results[0]
    
    report_df = pandas.DataFrame([build_summary_report(strategy)])
    report_str = report_df.to_string()
    
    return {"report": report_str}

def optimizer_node(state: OptimizationState):    
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", google_api_key=os.environ.get("GOOGLE_API_KEY"), temperature=0)
    
    prompt = f"""
    You are an expert algorithmic trading optimizer.
    
    Current Strategy Parameters: {state['params']}
    
    Backtest Results:
    {state['report']}
    
    Task: Analyze the performance. Suggest a NEW set of parameters for 'period_fast' and 'period_slow' to improve the Total Return.
    Constraints: 
    1. period_fast must be less than period_slow.
    2. Return ONLY a valid JSON object. Do not include markdown formatting or explanations.
    
    Example Output:
    {{"period_fast": 12, "period_slow": 26}}
    """
    
    response = llm.invoke(prompt)
    
    try:
        content = response.content.replace('```json', '').replace('```', '').strip()
        new_params = json.loads(content)
        print(f"--- Gemini Suggested New Params: {new_params} ---")
    except Exception as e:
        print(f"Error parsing LLM response: {e}. Keeping old params.")
        new_params = state['params']

    return {"params": new_params, "iteration": state['iteration'] + 1}

def should_continue(state: OptimizationState):
    if state['iteration'] < state['max_iterations']:
        return "optimizer"
    return END

# Build Graph
workflow = StateGraph(OptimizationState)
workflow.add_node("backtester", run_backtest_node)
workflow.add_node("optimizer", optimizer_node)

workflow.set_entry_point("backtester")
workflow.add_edge("optimizer", "backtester")
workflow.add_conditional_edges("backtester", should_continue)

app = workflow.compile()

# Run the Cycle
initial_params = {"period_fast": 10, "period_slow": 21}
inputs = {
    "params": initial_params, 
    "iteration": 0, 
    "max_iterations": 3, # Run 3 loops
    "report": ""
}

for output in app.stream(inputs):
    pass

Data download complete.

--- Running Backtest (Iter 0) with {'period_fast': 10, 'period_slow': 21} ---
--- Gemini Suggested New Params: {'period_fast': 5, 'period_slow': 13} ---

--- Running Backtest (Iter 1) with {'period_fast': 5, 'period_slow': 13} ---
--- Gemini Suggested New Params: {'period_fast': 10, 'period_slow': 30} ---

--- Running Backtest (Iter 2) with {'period_fast': 10, 'period_slow': 30} ---
--- Gemini Suggested New Params: {'period_fast': 8, 'period_slow': 25} ---

--- Running Backtest (Iter 3) with {'period_fast': 8, 'period_slow': 25} ---
